# shared

> We want to collect datacubes nonstop. Saving is a blocking operation which leaves gaps in the data we save on the order of a few seconds. The larger the datacube, the longer this gap. We really don't want these gaps, yet we want to save raw data. Saving raw data is extremely demanding (15 s of collect requires 15 s to save perhaps?). Can the saving be done in parallel? The following tries to address this issue.

In [ ]:
#| default_exp shared

# Shared `multiprocessing.Array`s for Continuous Camera Collect (Parallel DataCube Saving)

:::{.callout-tip}

This module can be imported using `from openhsi.shared import *`

:::

:::{.callout-warning}

Experimental

:::

In [ ]:
#| hide

# documentation extraction for class methods
from nbdev.showdoc import *

# unit tests using test_eq(...)
from fastcore.test import *

# monkey patching class methods using @patch
from fastcore.foundation import *
from fastcore.foundation import patch

# imitation of Julia's multiple dispatch using @typedispatch
from fastcore.dispatch import typedispatch

# bring forth **kwargs from an inherited class for documentation
from fastcore.meta import delegates

In [ ]:
#| export 

from fastcore.foundation import patch
from fastcore.meta import delegates
import numpy as np
import pandas as pd
import ctypes
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Iterable, Union, Callable, List, TypeVar, Generic, Tuple, Optional, Dict
from functools import reduce
from pathlib import Path
import xarray as xr

In [ ]:
#| export

from openhsi.data import *

from ctypes import c_int32, c_uint32, c_float
from multiprocessing import Process, Queue, Array

In [ ]:
#| export

class SharedCircArrayBuffer(CircArrayBuffer):
    """Circular FIFO Buffer implementation on multiprocessing.Array. Each put/get is a (n-1)darray."""
    
    def __init__(self, 
                 size:tuple = (100,100), # array buffer size
                 axis:int = 0,           # which axis to write along
                 c_dtype:type = c_int32, # C type for the array
                 show_func:Callable[[np.ndarray],"plot"] = None, # custom plotting function
                ):
        """Preallocate a array of `size` and type `c_dtype` and init write/read pointer. `c_dtype` needs to be from ctypes"""
        
        self.shared_data = Array(c_dtype, reduce(lambda x,y: x*y, size) )
        self.data = np.frombuffer(self.shared_data.get_obj(),dtype=c_dtype)
        self.data = self.data.reshape(size)
        
        self.size = size
        self.axis = axis
        self.write_pos = [slice(None,None,None) if i != axis else 0 for i in range(len(size)) ]
        self.read_pos  = self.write_pos.copy()
        self.slots_left = self.size[self.axis]
        self.show_func = show_func


In [ ]:
#| export

@delegates()
class SharedDataCube(CameraProperties):
    """Facilitates the collection, viewing, and saving of hyperspectral datacubes using
    two `SharedCircArrayBuffer`s that swap when save is called."""

    def __init__(self, 
                 n_lines:int = 16,        # number of along-track lines to buffer
                 processing_lvl:int = -1, # predefined processing recipe to use
                 **kwargs):
        """Preallocate array buffers"""
        self.n_lines = n_lines
        self.proc_lvl = processing_lvl
        super().__init__(**kwargs)
        self.set_processing_lvl(processing_lvl)
        self.dc_shape = (self.dc_shape[0],self.n_lines,self.dc_shape[1])
        self.dtype_out = c_int32 if self.dtype_out is np.int32 else self.dtype_out
        self.dtype_out = c_float if self.dtype_out is np.float32 else self.dtype_out
        
        # Only one set of buffers can be used at a time
        self.timestamps_swaps = [DateTimeBuffer(n_lines), DateTimeBuffer(n_lines)]
        self.dc_swaps         = [SharedCircArrayBuffer(size=self.dc_shape, axis=1, c_dtype=self.dtype_out),
                                 SharedCircArrayBuffer(size=self.dc_shape, axis=1, c_dtype=self.dtype_out)]
        print(f"Allocated {2*4*reduce(lambda x,y: x*y, self.dc_shape)/2**20:.02f} MB of RAM.")
        
        self.current_swap = 0
        self.timestamps   = self.timestamps_swaps[self.current_swap]
        self.dc           = self.dc_swaps[self.current_swap]
    
    def __repr__(self):
        return f"DataCube: shape = {self.dc_shape}, Processing level = {self.proc_lvl}\n"

    def put(self, x:np.ndarray):
        """Applies the composed tranforms and writes the 2D array into the data cube. Stores a timestamp for each push."""
        self.timestamps.update()
        self.dc.put( self.pipeline(x) )

In [ ]:
#| export

@patch
def save(self:SharedDataCube, 
         save_dir:str, # path to save directory
         preconfig_meta_path:str=None, # path to a json file with filled in metadata to copy
         prefix:str="", # prefix to use for the file name
         suffix:str="", # suffix to use for the file name
        ) -> Process:   # multiprocessing Process to wait on
    """Saves to a NetCDF file (and RGB representation) to directory dir_path in folder given by date with file name given by UTC time.
    Save is done in a separate multiprocess.Process."""
    if preconfig_meta_path is not None:
        with open(preconfig_meta_path) as json_file:
            attrs = json.load(json_file)
    else: attrs = {}
    
    self.directory = Path(f"{save_dir}/{self.timestamps[0].strftime('%Y_%m_%d')}/").mkdir(parents=True, exist_ok=True)
    self.directory = f"{save_dir}/{self.timestamps[0].strftime('%Y_%m_%d')}"
    
    wavelengths = self.binned_wavelengths if hasattr(self, "binned_wavelengths") else np.arange(self.dc.data.shape[2])
    
    if hasattr(self,"cam_temperatures"):
        self.coords = dict(wavelength=(["wavelength"],wavelengths),
                           x=(["x"],np.arange(self.dc.data.shape[0],dtype=np.int32)),
                           y=(["y"],np.arange(self.dc.data.shape[1],dtype=np.int32)),
                           time=(["time"],pd.to_datetime(self.timestamps.data,errors='coerce')),
                           temperature=(["temperature"],self.cam_temperatures.data))
    else:
        self.coords = dict(wavelength=(["wavelength"],wavelengths),
                           x=(["x"],np.arange(self.dc.data.shape[0],dtype=np.int32)),
                           y=(["y"],np.arange(self.dc.data.shape[1],dtype=np.int32)),
                           time=(["time"],pd.to_datetime(self.timestamps.data,errors='coerce')))
        
    fname = f"{self.directory}/{prefix}{self.timestamps[0].strftime('%Y_%m_%d-%H_%M_%S')}{suffix}"
    
    p = Process(target=save_shared_datacube, args=(fname,self.dc.shared_data,self.dtype_out,self.dc.size,self.coords,attrs,self.proc_lvl))
    p.start()
    print(f"Saving {fname} in another process.")
    
    self.current_swap = 0 if self.current_swap == 1 else 1
    self.timestamps   = self.timestamps_swaps[self.current_swap]
    self.dc           = self.dc_swaps[self.current_swap]
    if hasattr(self,"cam_temperatures"):
        self.cam_temperatures = self.cam_temps_swaps[self.current_swap]
    return p

In [ ]:
#| export

@patch
def show(self:SharedDataCube,
         plot_lib:str = "bokeh", # Plotting backend. This can be 'bokeh' or 'matplotlib'
         red_nm:float = 640.,    # Wavelength in nm to use as the red
         green_nm:float = 550.,  # Wavelength in nm to use as the green
         blue_nm:float = 470.,   # Wavelength in nm to use as the blue
         robust:Union[bool,int] = False, # Saturated linear stretch. E.g. setting `robust` to 2 will show the 2-98% percentile. Setting to `True` will default to `robust`=2. Robust to outliers
         hist_eq:bool = False,   # Choose to plot using histogram equilisation
         quick_imshow:bool = False, # Used to skip holoviews and use matplotlib for a static plot
        ) -> matplotlib.image: # bokeh or matplotlib plot
    """Generate an RGB image from chosen RGB wavelengths with histogram equalisation or percentile options.
    The plotting backend can be specified by `plot_lib` and can be "bokeh" or "matplotlib".
    Further customise your plot with `**plot_kwargs`. `quick_imshow` is used for saving figures quickly
    but cannot be used to make interactive plots. """

    rgb = np.zeros( (*self.dc.data.shape[:2],3), dtype=np.float32)
    if hasattr(self, "binned_wavelengths"):
        rgb[...,0] = self.dc.data[:,:,np.argmin(np.abs(self.binned_wavelengths-red_nm))]
        rgb[...,1] = self.dc.data[:,:,np.argmin(np.abs(self.binned_wavelengths-green_nm))]
        rgb[...,2] = self.dc.data[:,:,np.argmin(np.abs(self.binned_wavelengths-blue_nm))]
    else:
        rgb[...,0] = self.dc.data[:,:,int(self.dc.data.shape[2] / 2)]
        rgb[...,1] = self.dc.data[:,:,int(self.dc.data.shape[2] / 2)]
        rgb[...,2] = self.dc.data[:,:,int(self.dc.data.shape[2] / 2)]

    if robust and not hist_eq: # scale everything to the a saturated percentile
        if type(robust) is bool: robust = 2
        vmax = np.nanpercentile(rgb, 100-robust)
        vmin = np.nanpercentile(rgb, robust)
        rgb = ((rgb.astype("f8") - vmin) / (vmax - vmin)).astype("f4")
        rgb = np.minimum(np.maximum(rgb, 0), 1)
    elif hist_eq and not robust:
        img_hist, bins = np.histogram(rgb.flatten(), 256, density=True)
        cdf = img_hist.cumsum() # cumulative distribution function
        cdf = 1. * cdf / cdf[-1] # normalize
        img_eq = np.interp(rgb.flatten(), bins[:-1], cdf) # find new pixel values from linear interpolation of cdf
        rgb = img_eq.reshape(rgb.shape)
    elif robust and hist_eq:
        warnings.warn("Cannot mix robust with histogram equalisation. No RGB adjustments will be made.",stacklevel=2)
        rgb /= np.max(rgb)
    else:
        rgb /= np.max(rgb)

    if quick_imshow:
        fig, ax = plt.subplots(figsize=(12,3))
        ax.imshow(rgb,aspect="equal"); ax.set_xlabel("along-track"); ax.set_ylabel("cross-track")
        return fig

    import holoviews as hv
    hv.extension(plot_lib,logo=False)
    rgb_hv = hv.RGB((np.arange(rgb.shape[1]),np.arange(rgb.shape[0]),
                     rgb[:,:,0],rgb[:,:,1],rgb[:,:,2]))

    if plot_lib == "bokeh":
        return rgb_hv.opts(width=1000,height=250,frame_height=int(rgb.shape[0]//3)).opts(
            xlabel="along-track",ylabel="cross-track",invert_yaxis=True)
    else: # plot_lib == "matplotlib"
        return rgb_hv.opts(fig_inches=22).opts(
            xlabel="along-track",ylabel="cross-track",invert_yaxis=True)

In [ ]:
#| export

def save_shared_datacube(fname:str,          # NetCDF4 file name (without .nc)
                         shared_array:Array, # multiprocessing.Array shared array 
                         c_dtype:type,       # numpy data type
                         shape:Tuple,        # datacube numpy shape
                         coords_dict:Dict,   # coordinates dictionary
                         attrs_dict:Dict,    # metadata dictionary
                         proc_lvl:int,       # processing level used
                        ): 
    """Saves a NetCDF4 file given all the function parameters. Designed to be used with SharedOpenHSI which allocates a shared array."""
    
    data = np.frombuffer(shared_array.get_obj(),dtype=c_dtype)
    data = data.reshape(shape)

    nc = xr.Dataset(data_vars=dict(datacube=(["wavelength","x","y"],np.moveaxis(data, -1, 0) )),
                         coords=coords_dict, attrs=attrs_dict)  
    
    """provide metadata to NetCDF coordinates"""
    nc.x.attrs["long_name"]   = "cross-track"
    nc.x.attrs["units"]       = "pixels"
    nc.x.attrs["description"] = "cross-track spatial coordinates"
    nc.y.attrs["long_name"]   = "along-track"
    nc.y.attrs["units"]       = "pixels"
    nc.y.attrs["description"] = "along-track spatial coordinates"
    nc.time.attrs["long_name"]   = "along-track"
    nc.time.attrs["description"] = "along-track spatial coordinates"
    nc.wavelength.attrs["long_name"]   = "wavelength_nm"
    nc.wavelength.attrs["units"]       = "nanometers"
    nc.wavelength.attrs["description"] = "wavelength in nanometers."
    
    if "temperature" in coords_dict.keys():
        nc.temperature.attrs["long_name"] = "camera temperature"
        nc.temperature.attrs["units"] = "degrees Celsius"
        nc.temperature.attrs["description"] = "temperature of sensor at time of image capture"

    nc.datacube.attrs["long_name"]   = "hyperspectral datacube"
    nc.datacube.attrs["units"]       = "digital number"
    if proc_lvl in (4,5,7): nc.datacube.attrs["units"] = "uW/cm^2/sr/nm"
    elif proc_lvl in (6,8): nc.datacube.attrs["units"] = "percentage reflectance"
    nc.datacube.attrs["description"] = "hyperspectral datacube"
    
    nc.to_netcdf(fname+".nc")
    
    # quick save the histogram equalised RGB
    rgb = np.zeros( (*shape[:2],3), dtype=np.float32)
    rgb[...,0] = data[:,:,np.argmin(np.abs(coords_dict["wavelength"][1]-640.))]
    rgb[...,1] = data[:,:,np.argmin(np.abs(coords_dict["wavelength"][1]-550.))]
    rgb[...,2] = data[:,:,np.argmin(np.abs(coords_dict["wavelength"][1]-470.))]
    img_hist, bins = np.histogram(rgb.flatten(), 256, density=True)
    cdf = img_hist.cumsum() # cumulative distribution function
    cdf = 1. * cdf / cdf[-1] # normalize
    img_eq = np.interp(rgb.flatten(), bins[:-1], cdf) # find new pixel values from linear interpolation of cdf
    rgb = img_eq.reshape(rgb.shape)
    fig, ax = plt.subplots(figsize=(12,3))
    ax.imshow(rgb,aspect="equal"); ax.set_xlabel("along-track"); ax.set_ylabel("cross-track")
    fig.savefig(fname+".png",bbox_inches='tight', pad_inches=0)
    

## OpenHSI using shared multiprocessing.Array in SharedDataCube

`SharedOpenHSI` has the same API as `OpenHSI` with the addition of a camera temperature buffer that automatically swaps over when a save is called. 

In [ ]:
#| export

@delegates()
class SharedOpenHSI(SharedDataCube):
    """Base Class for the OpenHSI Camera."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        super().set_processing_lvl(self.proc_lvl)
        if callable(getattr(self,"get_temp",None)):
            self.cam_temps_swaps  = [CircArrayBuffer(size=(self.n_lines,),dtype=np.float32),
                                     CircArrayBuffer(size=(self.n_lines,),dtype=np.float32)]
            self.cam_temperatures = self.cam_temps_swaps[self.current_swap]
        
    def __enter__(self):
        return self
    
    def __close__(self):
        self.stop_cam()

    def __exit__(self, exc_type, exc_value, traceback):
        self.stop_cam()
        
    def collect(self):
        """Collect the hyperspectral datacube."""
        self.start_cam()
        for i in tqdm(range(self.n_lines)):
            self.put(self.get_img())
            
            if callable(getattr(self,"get_temp",None)):
                self.cam_temperatures.put( self.get_temp() )
        #self.stop_cam()
        
    def avgNimgs(self, n) -> np.ndarray:
        """Take `n` images and find the average"""
        data = np.zeros(tuple(self.settings['resolution'])+(n,),np.int32)
        
        self.start_cam()
        for f in range(n):
            data[:,:,f]=self.get_img()
        self.stop_cam()
        return np.mean(data,axis=2)

In [ ]:
show_doc(SharedOpenHSI.collect)

---

[source](https://github.com/YiweiMao/openhsi/blob/master/openhsi/shared.py#L273){target="_blank" style="float:right; font-size:smaller"}

### SharedOpenHSI.collect

>      SharedOpenHSI.collect ()

Collect the hyperspectral datacube.

In [ ]:
show_doc(SharedOpenHSI.avgNimgs)

---

[source](https://github.com/YiweiMao/openhsi/blob/master/openhsi/shared.py#L283){target="_blank" style="float:right; font-size:smaller"}

### SharedOpenHSI.avgNimgs

>      SharedOpenHSI.avgNimgs (n)

Take `n` images and find the average

See [https://openhsi.github.io/openhsi/capture.html](https://openhsi.github.io/openhsi/capture.html) for an example of how to use `SharedOpenHSI` for you custom cameras. 

### Shared FLIR Camera

This should work just like `openhsi.cameras.FlirCamera`. 

In [ ]:
#| export

@delegates()
class SharedFlirCamera(SharedOpenHSI):
    """Interface for FLIR camera"""
    
    def __init__(self, **kwargs):
        """Initialise FLIR camera"""
        super().__init__(**kwargs)
        
        try:
            from simple_pyspin import Camera
        except ModuleNotFoundError:
            warnings.warn("ModuleNotFoundError: No module named 'PySpin'.",stacklevel=2)
        
        self.flircam = Camera()
        self.flircam.init()
        self.flircam.GainAuto = 'Off'
        self.flircam.Gain = 0
        self.flircam.AcquisitionFrameRateAuto = 'Off'
        #self.flircam.AcquisitionFrameRateEnabled = True
        self.flircam.AcquisitionFrameRate = int( min(1_000/(self.settings["exposure_ms"]+1),120) )
    
        self.flircam.ExposureAuto = 'Off'
        self.flircam.ExposureTime = self.settings["exposure_ms"]*1e3 # convert to us
        self.flircam.GammaEnabled = False
        
        self.flircam.Width = self.flircam.SensorWidth if self.settings["win_resolution"][1] == 0 else self.settings["win_resolution"][1]
        self.flircam.Height = self.flircam.SensorHeight if self.settings["win_resolution"][0] == 0 else self.settings["win_resolution"][0]
        self.flircam.OffsetY, self.flircam.OffsetX = self.settings["win_offset"]

    
    def start_cam(self):
        self.flircam.start()
    
    def stop_cam(self):
        self.flircam.stop()
        
    def __close__(self):
        self.flircam.close()
    
    def get_img(self) -> np.ndarray:
        return self.flircam.get_array()
    
    def get_temp(self) -> float:
        return self.flircam.DeviceTemperature
    
    def set_exposure(self, exposure_ms:float):
        """sets the FLIR camera exposure time to `exposure_ms`."""
        self.settings["exposure_ms"] = exposure_ms
        
        self.flircam.AcquisitionFrameRateAuto = 'Off'
        #self.flircam.AcquisitionFrameRateEnabled = True
        self.flircam.AcquisitionFrameRate = int( min(1_000/(self.settings["exposure_ms"]+1),120) )
        self.flircam.ExposureAuto = 'Off'
        self.flircam.ExposureTime = self.settings["exposure_ms"]*1e3 # convert to us
        

For example, this could be used like so
```python
num_saved = 0
num2save  = 3 # save 3 datacubes

json_path = "../calibration_files/cam_settings_flir.json"
pkl_path  = "../calibration_files/cam_calibration_flir.pkl"

with SharedFlirCamera(n_lines=1280,processing_lvl=0,json_path=json_path,pkl_path=pkl_path) as cam:
    
    for i in range(num2save):
        
        cam.collect()
        print(f"collected from time: {cam.timestamps.data[0]} to {cam.timestamps.data[-1]}")
        
        if num_saved > 0:
            p.join() # wait for the last process to finish so we don't modify the data when it's being saved 
            pass
        
        p = cam.save("../temp")
        num_saved += 1
```

In [ ]:
#| export

from PIL import Image

## Parallel saving of datacubes while simulated camera is continuously running

Saving datacubes is a blocking operation but we want our camera to continue capturing while saving is taking place. This attempts to place the saving in another `multiprocessing.Process` and the underlying datacube is implemented as a shared `multiprocessing.Array`. 

:::{.callout-warning}

Experimental! However, the below example works! I'm a genious. Well, at very least, I feel like one for wrestling with the Global Interpreter Lock and coming out on top.

:::

In [ ]:
#| export

@delegates()
class SharedSimulatedCamera(SharedOpenHSI):
    """Simulated camera using an RGB image as an input. Hyperspectral data is produced using CIE XYZ matching functions."""
    def __init__(self, img_path:str = None, mode:str = None, **kwargs):
        """Initialise Simulated Camera"""
        super().__init__(**kwargs)
        self.mode = mode
        
        if img_path is None:
            self.img = np.random.randint(0,255,(*self.settings["resolution"],3))
        else:
            with Image.open(img_path) as img:
                img = img.resize((np.shape(img)[1],self.settings["resolution"][0]))
                self.img = np.array(img)[...,:3]
        
        if mode == "HgAr":
            self.gen = self.gen_sim_spectra()
        elif mode == "flat":
            self.gen = self.gen_flat()
        
        self.rgb_buff = CircArrayBuffer(self.img.shape,axis=1,dtype=np.uint8)
        self.rgb_buff.data = self.img
        self.rgb_buff.slots_left = 0 # make buffer full
        
        # Precompute the CIE XYZ matching functions to convert RGB values to a pseudo-spectra
        def piecewise_Guass(x,A,μ,σ1,σ2):
            t = (x-μ) / ( σ1 if x < μ else σ2 )
            return A * np.exp( -(t**2)/2 )
        def wavelength2xyz(λ):
            """λ is in nanometers"""
            λ *= 10 # convert to angstroms for the below formulas
            x̅ = piecewise_Guass(λ,  1.056, 5998, 379, 310) + \
                piecewise_Guass(λ,  0.362, 4420, 160, 267) + \
                piecewise_Guass(λ, -0.065, 5011, 204, 262)
            y̅ = piecewise_Guass(λ,  0.821, 5688, 469, 405) + \
                piecewise_Guass(λ,  0.286, 5309, 163, 311)
            z̅ = piecewise_Guass(λ,  1.217, 4370, 118, 360) + \
                piecewise_Guass(λ,  0.681, 4590, 260, 138)
            return np.array([x̅,y̅,z̅])
        self.λs = np.poly1d( np.polyfit(np.arange(len(self.calibration["wavelengths"])),self.calibration["wavelengths"] ,3) )(
                            np.arange(self.settings["resolution"][1]))
        self.xs = np.zeros( (1,len(self.λs)),dtype=np.float32)
        self.ys = self.xs.copy(); self.zs = self.xs.copy()
        for i in range(len(self.xs[0])):
            self.xs[0,i], self.ys[0,i], self.zs[0,i] = wavelength2xyz(self.λs[i])
        
        self.xyz_buff = CircArrayBuffer(self.settings["resolution"],axis=0,dtype=np.int32)
        
    def rgb2xyz_matching_funcs(self, rgb:np.ndarray) -> np.ndarray:
        """convert an RGB value to a pseudo-spectra with the CIE XYZ matching functions."""
        for i in range(rgb.shape[0]):
            self.xyz_buff.put( rgb[i,0]*self.xs + rgb[i,1]*self.ys + rgb[i,2]*self.zs )
        return self.xyz_buff.data

    
    def gen_flat(self):
        """simulated blackbody radiation"""
        T_K = 5800 # K. Sun's blackbody temperature
        # physical constants
        PLANCK_CONSTANT   = 6.62607015e-34 # J.s
        SPEED_OF_LIGHT    = 299_792_458    # m/s
        BOLTZMAN_CONSTANT = 1.38064852e-23 # J/K
        wavelengths = np.linspace(np.min(self.calibration["wavelengths"]),
                                  np.max(self.calibration["wavelengths"]),
                                  num=self.settings["resolution"][1])
        y = (2*PLANCK_CONSTANT*SPEED_OF_LIGHT**2)/(wavelengths*1e-9)**5 / (
                np.exp((PLANCK_CONSTANT*SPEED_OF_LIGHT)/
                       (wavelengths*1e-9*BOLTZMAN_CONSTANT*T_K)) - 1)
        y = np.uint8(255 * y/np.max(y))
        
        img = np.zeros(tuple(self.settings["resolution"]),dtype=np.uint8)
        for i in range(*self.settings["row_slice"]):
            img[i,:] = y
        while True:
            yield img
        
    def gen_sim_spectra(self):
        """simulated picture of a HgAr lamp"""
        lines_nm = [254,436,546,764,405,365,578,750,738,697,812,772,912,801,842,795,706,826,852,727] # approx sorted by emission strength
        img = np.zeros(tuple(self.settings["resolution"]),dtype=np.uint8)
        wavelengths = np.linspace(np.min(self.calibration["wavelengths"]),
                                  np.max(self.calibration["wavelengths"]),
                                  num=self.settings["resolution"][1])
        row_slice = slice(*self.settings["row_slice"])
        
        strength = 255
        for line in lines_nm: 
            indx = np.sum(wavelengths<line)
            if indx > 0 and indx < self.settings["resolution"][1]:
                img[row_slice,indx-2:indx+2] = strength
                strength -= 5
        while True:
            yield img
    
    def start_cam(self):
        pass
    
    def stop_cam(self):
        pass
    
    def get_img(self) -> np.ndarray:
        if self.mode in ("HgAr","flat"):
            return next(self.gen)
        if self.rgb_buff.is_empty():
            self.rgb_buff.slots_left = 0 # make buffer full again
        return self.rgb2xyz_matching_funcs(self.rgb_buff.get())
    
    def set_exposure(self):
        pass

    def get_temp(self):
        return 20.

In [ ]:
#| hardware
#| eval: false

num_saved = 0
num2save  = 3

with SharedSimulatedCamera(img_path="../../nbs/assets/great_hall_slide.png", n_lines=128, processing_lvl = 2, 
                     json_path="../../nbs/assets/cam_settings.json",pkl_path="../../nbs/assets/cam_calibration.pkl") as cam:
    
    for i in range(num2save):
        if num_saved > 0:
            #p.join() # waiting for the last process to finish will make this slow. 
            pass
            
        cam.collect()
        print(f"collected from time: {cam.timestamps.data[0]} to {cam.timestamps.data[-1]}")
        p = cam.save("../temp")
        num_saved += 1
    
    print(f"finished saving {num2save} datacubes")

Allocated 120.20 MB of RAM.


100%|███████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:03<00:00, 40.52it/s]

collected from time: 2022-09-21 07:27:25.292956 to 2022-09-21 07:27:28.422628


Saving ../temp/2022_09_21/2022_09_21-07_27_25 in another process.


100%|███████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:03<00:00, 35.29it/s]

collected from time: 2022-09-21 07:27:28.501124 to 2022-09-21 07:27:32.074323


Saving ../temp/2022_09_21/2022_09_21-07_27_28 in another process.


100%|███████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:04<00:00, 31.91it/s]

collected from time: 2022-09-21 07:27:32.144810 to 2022-09-21 07:27:36.105240


Saving ../temp/2022_09_21/2022_09_21-07_27_32 in another process.
finished saving 3 datacubes


Due to requiring double the amount of memory and more to facilitate saving in a separate process, make sure your datacubes can fit in your RAM. Have not tested this but I would suggest choosing `n_lines` <= 1/3 the amount used using the regular OpenHSI. 

In [ ]:
#| hardware
#| eval: false
#| hide

# remove the directory and file just created so it doesn't clog up the repo after running tests
import shutil, os
shutil.rmtree("../../nbs/"+[ f for f in os.listdir("../../nbs/") if "temp" in f][0])